# Step 1: Create an amplitude model

First, we `import` the necessary components of the expert system:

In [ ]:
from pycompwa.expertsystem.ui.system_control import (
    StateTransitionManager, InteractionTypes)
from pycompwa.expertsystem.amplitude.helicitydecay import (
    HelicityAmplitudeGeneratorXML)


def print_intermediate_states(solutions):
    """Just a little function to print the intermediate states"""
    from pycompwa.expertsystem.topology.graph import (
        get_intermediate_state_edges)
    print("intermediate states:")
    intermediate_states = set()
    for g in solutions:
        edge_id = get_intermediate_state_edges(g)[0]
        intermediate_states.add(g.edge_props[edge_id]['@Name'])
    print(intermediate_states)

## 1.1. Define the problem set

We first define the boundary conditions of our physics problem, such as initial state, final state, formalism type, etc. and pass all of that information to the `StateTransitionManager`. This is the main user interface class of the ComPWA expert system.

In [ ]:
initial_state = [("J/psi", [-1, 1])]
final_state = [("gamma", [-1, 1]), ("pi0", [0]), ("pi0", [0])]

tbd_manager = StateTransitionManager(initial_state, final_state,
                                     formalism_type='helicity',
                                     topology_building='isobar')

## 1.2. Prepare topologies

Create all topology graphs using the **isobar model** (tree of two-body decays) and initialize the graphs with the initial and final state. Remember that each interaction node defines its own set of conservation laws.

The `StateTransitionManager` (STM) defines three interaction types:

| Interaction          | Strength  |
| -------------------- | --------- |
| strong               | $60$      |
| electromagnetic (EM) | $1$       |
| weak                 | $10^{-4}$ |

By default, all three are used in the preparation stage. The function `prepare_graphs()` of the STM generates graphs with all possible combinations of interaction nodes. An overall interaction strength is assigned to each graph and they are grouped according to this strength.

In [ ]:
graph_interaction_settings_groups = tbd_manager.prepare_graphs()

## 1.3. Find solutions

If you are happy with the default settings generated by the ``StateTransitionManager``, just start with solving directly!

In [ ]:
solutions, violated_rules = tbd_manager.find_solutions(
    graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have a lot of solutions that are actually heavily suppressed (they involve two weak decays).

In general, you can modify the dictionary returned by `prepare_graphs()` directly, but the STM also comes with functionality to globally choose the allowed interaction types (`set_allowed_interaction_types()`).

So, go ahead and **disable** the **EM** and **weak** interaction!

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, violated_rules = tbd_manager.find_solutions(
    graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Huh, what happened here? Actually, since a **$\gamma$ particle** appears in one of the interaction nodes, the expert system knows that this node **must involve EM interactions**! Because the node can be an effective interaction, the weak interaction cannot be excluded, as it contains only a subset of conservation laws.

Since only the strong interaction was supposed to be used, this results in a warning and the STM automatically corrects the mistake.

Once the EM interaction is included, this warning disappears. Be aware, however, that the EM interaction is now available globally. Hence, there now might be solutions in which both nodes are electromagnetic.

In [ ]:
tbd_manager.set_allowed_interaction_types(
    [InteractionTypes.Strong, InteractionTypes.EM])
graph_interaction_settings_groups = tbd_manager.prepare_graphs()
solutions, violated_rules = tbd_manager.find_solutions(
    graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Great! Now we selected only the strongest contributions. Be aware, though, that there are more effects that can suppress certain decays. For example branching ratios. In this example $J/\Psi$ can decay into $\pi^0 + \rho^0$ or $\pi^0 + \omega$.

| decay                               | branching ratio |
| ----------------------------------- | --------------- |
| $$\omega \rightarrow \gamma+\pi^0$$ | 0.0828          |
| $$\rho^0 \rightarrow \gamma+\pi^0$$ | 0.0006          |

Unfortunately, the $\rho^0$ mainly decays into $\pi+\pi$, not $\gamma+\pi^0$ and is therefore suppressed. This information is currently not known to the expert system, but it is possible to hand the expert system a list of allowed intermediate states:

In [ ]:
# particles are found by name comparison,
# i.e. f2 will find all f2's and f all f's independent of their spin
tbd_manager.allowed_intermediate_particles = ['f']

solutions, violated_rules = tbd_manager.find_solutions(
    graph_interaction_settings_groups)

print("found " + str(len(solutions)) + " solutions!")
print_intermediate_states(solutions)

Now we have selected all amplitudes that involve **f** states. The appearing warnings notify the user that the solution space is only partial. For certain lines in the graph, no suitable particle was found (since only f states were allowed).

At this point, we are all set to generate some data using this amplitude model!

In [ ]:
xml_generator = HelicityAmplitudeGeneratorXML()
xml_generator.generate(solutions)
xml_generator.write_to_file('model.xml')

Have a look through the sections of the resulting ``model.xml`` file to see what you recognize from the problem set defined so far.

In what follows, we reuse this file in each step. This means you don't have to rerun Step 1 each time you don't do an analysis.